# AI Learn X 경연 - SSD 신뢰성 사전 불량 예측


**AI Learn X 경연대회 - SSD 신뢰성 사전 불량 예측**에 참가하신 여러분 모두 환영합니다.

본 자료는 경연대회에 참여를 원하시나 시작을 어려워하시는 분들을 대상으로 제공되는 기본 코드입니다. 따라서 **본인이 원하는 대로 자유롭게 수정하시거나, 아예 사용하지 않아도 무방합니다.**

기본으로 제공되는 본 코드는 플랫폼 상에서 데이터를 불러오는 방법부터 간단한 전처리 및 딥러닝 모델링의 모든 과정을 소개하고 있습니다. 나아가서 작성한 코드의 결과를 플랫폼 상에서 바로 채점하기 위한 과정까지 확인해볼 수 있습니다.

In [ ]:
import os
import sys

import pandas as pd
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

import tqdm
import numpy as np
import random

import warnings
warnings.filterwarnings("ignore")

data_dir = '/mnt/elice/dataset'

In [ ]:
# 재현성 확보를 위해 시드값을 고정합니다.
seed = 42
np.random.seed(seed)
random.seed(seed)

## 데이터 불러오기

학습 데이터와 테스트 데이터를 불러옵니다.
- Info :
    - X 변수들은 SSD신뢰성에 영향을 줄 수 있는 SMART Attribute (비식별화된 상태)
    - Y는 특정 기준에 따라 분류된 Pass(0)/Fail(1)
    - 한 Serial Number에 대해 여러 개의 Row가 존재하지만 Y 값은 Serial Number 별로 동일한 값을 가짐
    - X 변수는 18개

|Dataset|Length|#Normal|#Abnormal|#Serial|#Normal Serial|#Abnormal Serial|
|----|----|----|----|----|----|----|
|train|475,467|475,467|0|7,060|7,060|0|
|test|218,869|118,768|100,101|3,281|1,765|1,516|

|Index|Features|Format|Description|
|----|----|----|:----|
|1|Serial Number|15|Serial Number|
|2|TIMESTAMP|2020.2.9  4:59:00 AM|Date-time reference|
|3|X1|EI83N072710203N8D|Equipment name|
|4|X2~18|27|Features|

In [ ]:
# train_df: serial number, timestamp, X1, X2~18, Y
train_df = pd.read_csv(os.path.join(data_dir, "train.csv"), index_col='Serial Number')
# test_x: serial number, timestamp, X1, X2~18
test_x = pd.read_csv(os.path.join(data_dir, "test_x.csv"), index_col='Serial Number')

''' timestamp 열 형식 바꾸기 '''
train_df['TIMESTAMP'] = pd.to_datetime(train_df['TIMESTAMP'])
train_df['TIMESTAMP'] = train_df['TIMESTAMP'].map(lambda t: t.strftime('%Y-%m-%d %H:%M'))
test_x['TIMESTAMP'] = pd.to_datetime(test_x['TIMESTAMP'])
test_x['TIMESTAMP'] = test_x['TIMESTAMP'].map(lambda t: t.strftime('%Y-%m-%d %H:%M'))

''' 컬럼 키 추출 '''
serial_key = train_df.index.name
date_time_key = list(train_df.columns)[0]
feature_keys = list(train_df.columns)[2:-1]
target_key = list(train_df.columns)[-1]

# train_x: serial number, timestamp, X1, X2~18
train_x = train_df.drop(columns='Y')
# train_y: serial_number, Y
train_y = pd.read_csv(os.path.join(data_dir, "train_y.csv"), index_col='Serial Number')

In [ ]:
print(serial_key)
print(date_time_key)
print(feature_keys)
print(target_key)

Serial Number
TIMESTAMP
['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18']
Y


불러온 학습 데이터의 일부를 확인해봅시다.

In [ ]:
train_x.head()

,TIMESTAMP,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
Serial Number,,,,,,,,,,,,,,,,,,,
19,2020-02-09 16:24,EI83N072710203N8H,22,18,22,16,32,99,91219,43095,2132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,2020-02-10 16:39,EI83N072710203N8H,22,18,22,16,32,99,91413,43214,2133,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,2020-02-11 16:54,EI83N072710203N8H,22,18,22,16,32,99,91606,43332,2135,49.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
19,2020-02-12 17:09,EI83N072710203N8H,22,18,22,16,32,99,91799,43449,2136,73.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0
19,2020-02-13 17:24,EI83N072710203N8H,22,18,23,16,32,99,91991,43566,2137,97.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0


불러온 평가용 데이터의 일부를 확인해봅시다.

In [ ]:
test_x.head()

,TIMESTAMP,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
Serial Number,,,,,,,,,,,,,,,,,,,
100122I,2020-02-09 13:08,EJ86N538510606DC8,76,74,22,12,30,99,62160,27565,1980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100122I,2020-02-10 13:23,EJ86N538510606DC8,76,74,22,12,30,99,62332,27652,1983,24.0,20.0,0.0,0.0,20.0,0.0,0.0,0.0
100122I,2020-02-11 13:38,EJ86N538510606DC8,76,74,22,12,30,99,62501,27737,1985,48.0,30.0,0.0,0.0,30.0,0.0,0.0,0.0
100122I,2020-02-12 13:53,EJ86N538510606DC8,76,74,22,12,30,99,62670,27819,1988,73.0,43.0,0.0,0.0,43.0,0.0,0.0,0.0
100122I,2020-02-13 14:08,EJ86N538510606DC8,76,74,22,12,30,99,62843,27906,1989,97.0,61.0,0.0,0.0,61.0,0.0,0.0,0.0


각 Serial Number 별 Y 값의 일부를 확인해봅시다.

In [ ]:
train_y.head()

,Y
Serial Number,
19,1
42,1
48,1
51,1
67,1


데이터의 개수와 Unique한 Serial Number의 개수를 확인해봅시다.

In [ ]:
print("Train Data 크기 :", train_x.shape)
print("Test Data 크기 :", test_x.shape)

# Serial Number의 unique 값과 전체 데이터의 수를 비교합니다.
print("Train Data의 Serial Number의 unique 값 :", len(train_x.index.unique()))
print("Test Data의 Serial Number의 unique 값 :", len(test_x.index.unique()))

Train Data 크기 : (555456, 19)
Test Data 크기 : (138880, 19)
Train Data의 Serial Number의 unique 값 : 8272
Test Data의 Serial Number의 unique 값 : 2069


## 데이터 전처리

장비 이름을 나타내는 X1 변수를 제거하고, 학습 데이터와 테스트 데이터를 표준화 합니다.<br>

경연대회를 진행하면서 이 전처리 기법 뿐만 아니라 다양한 전처리 기법을 활용하실 수 있습니다.

장비 이름을 나타내는 X1 변수를 제거합니다.

In [ ]:
train_x.drop(columns='X1', inplace=True)
test_x.drop(columns='X1', inplace=True)

`StandardScaler` 를 활용해 표준화합니다.

In [ ]:
# 표준화를 위한 StandardScaler 객체 생성
scaler = StandardScaler()

# 학습 데이터에 대해 스케일러 학습 및 정규화
# fit_transform()을 사용하여 학습 데이터에 대해 학습과 변환을 동시에 수행
train_x[feature_keys] = scaler.fit_transform(train_df[feature_keys])

# 테스트 데이터에 대해 학습 데이터를 기반으로 정규화
test_x[feature_keys] = scaler.transform(test_x[feature_keys])

표준화 완료한 데이터를 Serial Number를 기준으로 분리합니다.

In [ ]:
# 데이터를 index가 같은 컬럼 별로 분리해서 리스트에 저장
# group: (그룹 이름, 그룹 데이터프레임) => group[1]: 그룹 데이터프레임
train_x_by_serial = [group[1] for group in train_x.groupby(train_x.index)]
test_x_by_serial = [group[1] for group in test_x.groupby(test_x.index)]

# TIMESTAMP 값을 기준으로 정렬
train_x_by_serial = [group.sort_values('TIMESTAMP') for group in train_x_by_serial]
test_x_by_serial = [group.sort_values('TIMESTAMP') for group in test_x_by_serial]

In [ ]:
train_x_by_serial[2]

,TIMESTAMP,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
Serial Number,,,,,,,,,,,,,,,,,,
48,2020-02-09 08:01,0.237359,0.176644,1.060111,1.067976,1.258588,-0.079582,0.845695,0.629975,0.059574,-1.734490,-0.465737,-0.012637,-0.035175,-0.464819,-0.010677,-0.011952,-0.013366
48,2020-02-10 08:16,0.237359,0.176644,1.060111,1.067976,1.258588,-0.079582,0.849884,0.633251,0.059664,-1.684896,-0.410967,-0.012621,-0.035175,-0.409868,-0.010677,-0.011952,-0.012643
48,2020-02-11 08:31,0.237359,0.176644,1.060111,1.067976,1.258588,-0.079582,0.854143,0.636589,0.059844,-1.635302,-0.315119,-0.012613,-0.035175,-0.313703,-0.010677,-0.011952,-0.012282
48,2020-02-12 08:46,0.237359,0.176644,1.060111,1.067976,1.258588,-0.079582,0.858803,0.640521,0.060115,-1.583642,-0.219271,-0.012589,-0.035175,-0.217539,-0.010677,-0.011952,-0.011199
48,2020-02-13 09:01,0.237359,0.176644,1.060111,1.067976,1.258588,-0.079582,0.863227,0.644140,0.060295,-1.534048,-0.127987,-0.012589,-0.035175,-0.125954,-0.010677,-0.011952,-0.011199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,2020-04-14 00:01,0.237359,0.176644,1.407310,1.067976,1.258588,-0.079582,1.133848,0.867393,0.078570,1.480844,4.029984,-0.012373,-0.035175,4.045745,-0.010677,-0.011952,-0.001446
48,2020-04-15 00:16,0.237359,0.176644,1.407310,1.067976,1.258588,-0.079582,1.138296,0.870981,0.078840,1.530438,4.198859,-0.012345,-0.035175,4.215177,-0.010677,-0.011952,-0.000362
48,2020-04-16 00:31,0.237359,0.176644,1.407310,1.067976,1.258588,-0.079582,1.142649,0.874445,0.079110,1.580032,4.317527,-0.012345,-0.035175,4.334238,-0.010677,-0.011952,-0.000362


In [ ]:
train_x_by_serial[2].index.unique()

Int64Index([48], dtype='int64', name='Serial Number')

In [ ]:
train_x_by_serial[2].index.unique().item()

48

In [ ]:
train_y.loc[54509]

Y    0
Name: 54509, dtype: int64

In [ ]:
train_y.value_counts()

Y
0    7060
1    1212
dtype: int64

## 데이터셋 생성

학습용 데이터셋과 검증용 데이터셋을 생성합니다. 데이터의 Serial Number를 기준으로 8:2 비율로 분할하고, 동시에 학습용 데이터셋과 검증용 데이터셋의 Y 비율이 Serial Number 수 기준으로 비슷하도록 분할합니다.

In [ ]:
def train_test_split(Xs, ys, test_ratio=0.2):
    ''' 각 (x, y) 쌍을 label 별로 딕셔너리에 저장 '''
    data_per_label = {}

    for x, y in zip(Xs, ys):
        label = y
        if label not in data_per_label:
            data_per_label[label] = []
        # key: label, value: 해당 label에 해당하는 (x, y) 쌍의 리스트
        data_per_label[label].append((x, y))

    train = []
    test = []

    for label in data_per_label:
        # label에 해당하는 데이터 가져온다
        data = data_per_label[label]
        # 테스트 데이터의 개수
        n_test = int(len(data) * test_ratio)
        test += data[:n_test]
        train += data[n_test:]

    X_train, y_train = zip(*train)
    X_test, y_test = zip(*test)

    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_x_by_serial, train_y['Y'], test_ratio=0.2)

# X data에서 Timestamp를 제거합니다.
X_train = [x.drop(columns='TIMESTAMP') for x in X_train]
X_val = [x.drop(columns='TIMESTAMP') for x in X_val]
X_test = [x.drop(columns='TIMESTAMP') for x in test_x_by_serial]

print("Train Data의 개수 :", len(X_train))
print("Validation Data의 개수 :", len(X_val))
print("Test Data의 개수 :", len(X_test))

Train Data의 개수 : 6618
Validation Data의 개수 : 1654
Test Data의 개수 : 2069


머신러닝 모델에 적용하기 위해 학습, 검증, 테스트용 데이터를 각각 하나의 numpy array로 합칩니다.

In [ ]:
def align_data(data, series_length):
    # X2~18열에 해당하는 시계열 데이터 값만 추출
    data_features = [x[feature_keys] for x in data]
    len_data = len(data_features)
    length_aligned_X = []
    for x in data_features:
        # 시계열 데이터 길이가 series_length 이상이면 > 뒷부분 잘라냄
        if len(x) >= series_length:
            length_aligned_X.append(x[:series_length])
        # 시계열 데이터 길이가 series_length보다 작으면 > 마지막 행을 반복하여 길이를 맞춤
        else:
            length_aligned_X.append(x.append([x.iloc[-1]] * (series_length - len(x))))
    return np.array(length_aligned_X).reshape(len_data, -1)

In [ ]:
series_length = 50

X_train = align_data(X_train, series_length)
X_val = align_data(X_val, series_length)
X_test = align_data(X_test, series_length)

In [ ]:
17 * 50

850

In [ ]:
len(X_train[0])

850

y data도 numpy array로 변환합니다.

In [ ]:
y_train = np.array(y_train)
y_val = np.array(y_val)

## 모델 학습

모델은 각자 자유롭게 구성할 수 있으며 올바른 형식으로 `submission.csv`를 쓴다면 다양한 모델이 허용됩니다.

아래는 이해를 돕기 위한 예시 코드로 간단한 의사 결정 트리 머신러닝 모델을 학습하는 과정입니다.

In [ ]:
# Scikit-learn을 활용해서 시계열 데이터 분류 모델을 학습합니다.
# RandomForestClassifier 객체 생성
clf = DecisionTreeClassifier(random_state=42)

# 학습 데이터에 대해 학습
clf.fit(X_train, y_train)

# 검증 데이터에 대해 예측
y_val_pred = clf.predict(X_val)

# 검증 데이터에 대한 F1 Score 계산
f1 = f1_score(y_val, y_val_pred, average='macro')

print(f'검증 데이터에 대한 F1 Score: {f1:.4f}')

검증 데이터에 대한 F1 Score: 0.7776


## 모델 평가 및 결과 저장

분류 모델을 평가하기 위한 지표로는 대표적으로 **정확도(Accuracy)** 가 있습니다. 하지만 본 경연대회에서는 정확도만으로 온전히 평가할 수 없는 데이터 상의 특징이 있습니다.

따라서, **F1 score** 지표로 순위를 결정할 것입니다.

여러분들이 작성하신 모델의 성능을 평가하기 위해서는, 지시사항에 나와 있는 대로 **평가 데이터**를 불러와서 예측을 수행하고, 그 결과를 반드시 `submission.csv` 파일에 기록하여 제출해야 합니다.

In [ ]:
submission = pd.read_csv(os.path.join(data_dir, "test_y.csv"), index_col='Serial Number')

# 테스트 데이터에 대해 예측하고, 결과를 저장합니다.
y_test_pred = clf.predict(X_test)
submission["Y"] = y_test_pred
submission.to_csv("submission.csv", index_label='Serial Number')

### 결과 검증

`submission.csv` 파일을 다시 불러와 올바르게 값을 채웠는지 다시 한번 확인합니다.

In [ ]:
submission = pd.read_csv("submission.csv", index_col='Serial Number')
submission

,Y
Serial Number,
100122I,1
100368G,0
101403L,0
101426G,0
101505B,0
...,...
997719U,0
998737L,0
999308S,0


In [ ]:
# 예측 결과 중 1의 비율을 계산하고, 학습용 데이터의 비율과 비교합니다.
print(submission["Y"].mean())
print(train_y.mean())

0.21362977283711937
Y    0.146518
dtype: float64


### 제출

우측 상단의 제출 버튼을 눌러, `competition.ipynb` 파일과 `submission.csv` 파일을 제출합니다.